In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .getOrCreate()
spark

In [7]:
# from pyspark.sql import SparkSession
# from pyspark.sql import functions as F
# from pyspark.sql.types import DoubleType

# # Initialize Spark session
# spark = SparkSession.builder \
#     .appName("CompanyGrowthDF") \
#     .master("local[*]") \
#     .getOrCreate()

# # Step 1: Read CSV file
# df = spark.read.option("header", "true").csv(r"C:\Users\ssardana\OneDrive - Capgemini\Desktop\OneDrive_1_23-6-2025 (1)\Industry_Analysis_Src01.csv")

# # Debug: Check schema and sample data
# df.printSchema()
# df.show(5, truncate=False)

# # Step 2: Clean and transform columns
# df_clean = df.withColumn("Market_Capital", F.col("Market_Capital(in Million)").cast(DoubleType())) \
#     .withColumn("Formatted_Date", F.to_date(F.col("Date"), "dd-MM-yyyy")) \
#     .withColumn("Year", F.year(F.col("Formatted_Date"))) \
#     .select("Company Name", "Year", "Market_Capital")

# # Debug: Verify cleaned data
# df_clean.show(5, truncate=False)

# # Step 3: Filter for years and non-null Market_Capital
# df_filtered = df_clean.filter(
#     (F.col("Year").isin(2009, 2010, 2011)) & F.col("Market_Capital").isNotNull()
# )

# # Step 4: Compute total Market Capital per (Company, Year)
# agg_df = df_filtered.groupBy("Company Name", "Year") \
#     .agg(F.round(F.sum("Market_Capital"), 2).alias("Total_Market_Capital"))

# # Step 5: Pivot year columns into separate fields
# pivoted = agg_df.groupBy("Company Name").pivot("Year", [2009, 2010, 2011]) \
#     .agg(F.first("Total_Market_Capital"))

# # Step 6: Compute Growth from 2009 to 2011
# result = pivoted.withColumn(
#     "Growth_2009_to_2011",
#     F.round(((F.col("2011") - F.col("2009")) / F.col("2009")) * 100, 2)
# ).select("Company Name", "2009", "2011", "Growth_2009_to_2011")

# # Final Output
# result.orderBy("Company Name").show(truncate=False)

# # Stop Spark session
# spark.stop()


from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("CompanyGrowthByQuarter") \
    .master("local[*]") \
    .getOrCreate()

# Load the dataset
df = spark.read.option("header", "true").csv("C:\Users\ssardana\OneDrive - Capgemini\Desktop\OneDrive_1_23-6-2025 (1)\Industry_Analysis_Src01.csv")

# Clean and transform relevant columns
df_clean = df.withColumn("Market_Capital", F.col("Market_Capital(in Million)").cast(DoubleType())) \
             .select("Company Name", "Quarter", "Market_Capital")

# Filter out rows with nulls and focus on Q1 and Q4
df_filtered = df_clean.filter(F.col("Market_Capital").isNotNull() & F.col("Quarter").isin("Q1", "Q4"))

# Aggregate total market capital per company per quarter
agg_df = df_filtered.groupBy("Company Name", "Quarter") \
                    .agg(F.round(F.sum("Market_Capital"), 2).alias("Total_Market_Capital"))

# Pivot to get Q1 and Q4 in separate columns
pivoted = agg_df.groupBy("Company Name").pivot("Quarter", ["Q1", "Q4"]) \
                .agg(F.first("Total_Market_Capital"))

# Calculate growth from Q1 to Q4
result = pivoted.withColumn(
    "Growth_Q1_to_Q4",
    F.round(((F.col("Q4") - F.col("Q1")) / F.col("Q1")) * 100, 2)
).select("Company Name", "Q1", "Q4", "Growth_Q1_to_Q4")

# Show the result
result.orderBy("Company Name").show(truncate=False)

# Stop Spark session
spark.stop()


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3113300839.py, line 64)

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("CompanyGrowthByQuarter") \
    .master("local[*]") \
    .getOrCreate()

# Load the dataset using raw string to avoid unicode escape errors
df = spark.read.option("header", "true").csv(r"C:\Users\ssardana\OneDrive - Capgemini\Desktop\OneDrive_1_23-6-2025 (1)\Industry_Analysis_Src01.csv")

# Clean and transform relevant columns
df_clean = df.withColumn("Market_Capital", F.col("Market_Capital(in Million)").cast(DoubleType())) \
             .select("Company Name", "Quarter", "Market_Capital")

# Filter out rows with nulls and focus on Q1 and Q4
df_filtered = df_clean.filter(F.col("Market_Capital").isNotNull() & F.col("Quarter").isin("Q1", "Q4"))

# Aggregate total market capital per company per quarter
agg_df = df_filtered.groupBy("Company Name", "Quarter") \
                    .agg(F.round(F.sum("Market_Capital"), 2).alias("Total_Market_Capital"))

# Pivot to get Q1 and Q4 in separate columns
pivoted = agg_df.groupBy("Company Name").pivot("Quarter", ["Q1", "Q4"]) \
                .agg(F.first("Total_Market_Capital"))

# Calculate growth from Q1 to Q4
result = pivoted.withColumn(
    "Growth_Q1_to_Q4",
    F.round(((F.col("Q4") - F.col("Q1")) / F.col("Q1")) * 100, 2)
).select("Company Name", "Q1", "Q4", "Growth_Q1_to_Q4")

# Show the result
result.orderBy("Company Name").show(truncate=False)

# Stop Spark session
spark.stop()

+------------------------------+--------+--------+---------------+
|Company Name                  |Q1      |Q4      |Growth_Q1_to_Q4|
+------------------------------+--------+--------+---------------+
|Aedes                         |7684.8  |7683.0  |-0.02          |
|Allied Properties (HK) Ltd.   |24425.3 |23042.0 |-5.66          |
|Anna Infrastructures Ltd      |5873.7  |5954.7  |1.38           |
|Arihant Foundations & Hou     |7642.8  |7733.0  |1.18           |
|Aspen Group Ltd               |7669.8  |8066.8  |5.18           |
|Audi AG                       |24308.6 |30776.6 |26.61          |
|BSEL Infrastructure Realty Ltd|7652.0  |7669.8  |0.23           |
|BYD Co Ltd                    |12993.2 |14019.4 |7.9            |
|Bharat Petroleum Corp Ltd     |4885.8  |5051.0  |3.38           |
|Brookfield Asset              |3937.8  |4184.3  |6.26           |
|CLP Holdings Limited          |6107.0  |6344.5  |3.89           |
|Cognizant                     |7227.0  |7545.0  |4.4         

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("CompanyQuarterlyGrowth") \
    .master("local[*]") \
    .getOrCreate()

# Load the dataset
df = spark.read.option("header", "true").csv(r"C:\Users\ssardana\OneDrive - Capgemini\Desktop\OneDrive_1_23-6-2025 (1)\Industry_Analysis_Src01.csv")

# Convert Market Capital to numeric and extract year from Date
df_clean = df.withColumn("Market_Capital", F.col("Market_Capital(in Million)").cast(DoubleType())) \
             .withColumn("ParsedDate", F.to_date(F.col("Date"), "d-MMM-yy")) \
             .withColumn("Year", F.year("ParsedDate")) \
             .select("Company Name", "Year", "Quarter", "Market_Capital")

# Filter out rows with nulls
df_filtered = df_clean.filter(F.col("Market_Capital").isNotNull())

# Aggregate total market capital per company per year and quarter
agg_df = df_filtered.groupBy("Company Name", "Year", "Quarter") \
                    .agg(F.round(F.sum("Market_Capital"), 2).alias("Total_Market_Capital"))

# Pivot to get Q1, Q2, Q3, Q4 in separate columns
pivoted = agg_df.groupBy("Company Name", "Year").pivot("Quarter", ["Q1", "Q2", "Q3", "Q4"]) \
                .agg(F.first("Total_Market_Capital"))

# Calculate growth from Q1 to Q4 within each year
result = pivoted.withColumn(
    "Growth_Q1_to_Q4",
    F.round(((F.col("Q4") - F.col("Q1")) / F.col("Q1")) * 100, 2)
).select("Company Name", "Year", "Q1", "Q2", "Q3", "Q4", "Growth_Q1_to_Q4")

# Show the result
result.orderBy("Company Name", "Year").show(truncate=False)

# Stop Spark session
spark.stop()


+------------------------------+----+------+------+------+-------+---------------+
|Company Name                  |Year|Q1    |Q2    |Q3    |Q4     |Growth_Q1_to_Q4|
+------------------------------+----+------+------+------+-------+---------------+
|Aedes                         |2009|2543.8|2561.0|2591.0|2513.0 |-1.21          |
|Aedes                         |2010|2555.0|2532.0|2600.0|2584.0 |1.14           |
|Aedes                         |2011|2586.0|2510.8|2584.0|2586.0 |0.0            |
|Allied Properties (HK) Ltd.   |2009|7792.3|8441.0|7099.0|8048.0 |3.28           |
|Allied Properties (HK) Ltd.   |2010|8694.0|6837.1|7977.8|7497.0 |-13.77         |
|Allied Properties (HK) Ltd.   |2011|7939.0|7280.8|7977.8|7497.0 |-5.57          |
|Anna Infrastructures Ltd      |2009|1964.7|1968.0|2028.0|1942.0 |-1.16          |
|Anna Infrastructures Ltd      |2010|1967.0|1972.0|1968.0|2028.0 |3.1            |
|Anna Infrastructures Ltd      |2011|1942.0|1972.0|1968.0|1984.7 |2.2            |
|Ari